In [42]:
import pickle
from scipy.sparse import csr_matrix
import pandas as pd

# Load the saved ALS model from "../model" folder
def load_model():
    with open('../model/als_model.pkl', 'rb') as f:
        return pickle.load(f)

# Load the interaction matrix, customer_map, and product_map from "../model" folder
def load_data():
    # Load the interaction matrix and convert it to CSR format for fast row slicing
    with open('../model/interaction_matrix.pkl', 'rb') as f:
        interaction_matrix = pickle.load(f).tocsr()  # Convert to CSR format

    with open('../model/customer_map.pkl', 'rb') as f:
        customer_map = pickle.load(f)

    with open('../model/product_map.pkl', 'rb') as f:
        product_map = pickle.load(f)

    return interaction_matrix, customer_map, product_map

# Function to recommend products for a given customer ID
def recommend_products(customer_index, model, interaction_matrix, product_map, num_recommendations=10):
    print('customer_index', customer_index)
    
    if customer_index is None:
        return "Customer not found"

    user_interaction = interaction_matrix[0]

    # Get recommendations using the ALS model
    recommendations = model.recommend(customer_index, user_interaction, N=num_recommendations)
    
    # Filter out invalid recommendations and map back the recommended product indices to product IDs
    recommended_product_ids = []
    for product_idx in recommendations[0]:
        # Ensure the product index is valid and exists in product_map
        if product_idx in product_map:
            recommended_product_ids.append(product_map[product_idx])
        else:
            print(f"Invalid product index: {product_idx}")

    return recommended_product_ids if recommended_product_ids else "No valid recommendations"

# Example usage
als_model = load_model()
interaction_matrix, customer_map, product_map = load_data()

# Assuming you have a valid customer_index (you can get this from customer_map based on the UUID)
customer_index = 0  # Example index, replace with actual customer index
recommended_products = recommend_products(customer_index, als_model, interaction_matrix, product_map)
print(f"Recommended products: {recommended_products}")


customer_index 0
Recommended products: [7654321, 9876542, 5928374, 1948273, 3182746, 4728391, 2365812, 8374627, 8374629, 6283742]


In [46]:
recommend_products(100, als_model, interaction_matrix, product_map, 30)

customer_index 100


[2178395,
 4827395,
 1948273,
 1827364,
 5937482,
 9126487,
 1928314,
 9876542,
 5283741,
 5896387,
 5927381,
 6382749,
 1628374,
 9374521,
 9512362,
 4827359,
 6372819,
 6173829,
 7412583,
 8293746,
 1596389,
 4172958,
 1827395,
 7162539,
 2635841,
 2837462,
 8471926,
 7293856,
 7293846,
 7361829]

In [36]:
list(customer_map.values())[1]

2

In [25]:
customer_map

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20,
 20: 21,
 21: 22,
 22: 23,
 23: 24,
 24: 25,
 25: 26,
 26: 27,
 27: 28,
 28: 29,
 29: 30,
 30: 31,
 31: 32,
 32: 33,
 33: 34,
 34: 35,
 35: 36,
 36: 37,
 37: 38,
 38: 39,
 39: 40,
 40: 41,
 41: 42,
 42: 43,
 43: 44,
 44: 45,
 45: 46,
 46: 47,
 47: 48,
 48: 49,
 49: 50,
 50: 51,
 51: 52,
 52: 53,
 53: 54,
 54: 55,
 55: 56,
 56: 57,
 57: 58,
 58: 59,
 59: 60,
 60: 61,
 61: 62,
 62: 63,
 63: 64,
 64: 65,
 65: 66,
 66: 67,
 67: 68,
 68: 69,
 69: 70,
 70: 71,
 71: 72,
 72: 73,
 73: 74,
 74: 75,
 75: 76,
 76: 77,
 77: 78,
 78: 79,
 79: 80,
 80: 81,
 81: 82,
 82: 83,
 83: 84,
 84: 85,
 85: 86,
 86: 87,
 87: 88,
 88: 89,
 89: 90,
 90: 91,
 91: 92,
 92: 93,
 93: 94,
 94: 95,
 95: 96,
 96: 97,
 97: 98,
 98: 99,
 99: 100,
 100: 101,
 101: 102,
 102: 103,
 103: 104,
 104: 105,
 105: 106,
 106: 107,
 107: 108,
 108: 109,
 109: 110,
 110: 11